In [ ]:
from fbprophet import Prophet


In [ ]:
import urllib.request
import pandas as pd
import json
import requests
import datetime as dt
import plotly.express as px

**Alphavantage API Endpoint**

* Documentation: https://www.alphavantage.co/documentation/



In [ ]:
base_url = 'https://www.alphavantage.co/query?'

**The Parameters (According to Documentation)**

What you will also need: to signup to Alphavantage and get your API key and paste the API key where it says 'YOUR_API_KEY'

This code asks for the stock price of Google (with ticker symbol: GOOGL)

In [ ]:
# parameters
params = {
    'function': 'TIME_SERIES_DAILY',
    'symbol': 'GOOGL',
    'apikey': 'YOUR_API_KEY'
}

**Get Data and Store the Response**

In [ ]:
response = requests.get(base_url, params=params)

**Examine JSON Datastructure in Response**

In [ ]:
print(response.json())

**Stock Prices**

* In this part of the JSON: 'Time Series (Daily)':

In [ ]:
data = response.json()

Save the JSON file with filename 'data.json'

Notes: that location of where to save the file must be included. For example, '/content/drive/MyDrive/Datasets/data.json' saves 'data.json' in Google Drive with folder '/content/drive/MyDrive/Datasets/'

In [ ]:
with open('/content/drive/MyDrive/Datasets/data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

**Get that part of the JSON**

In [ ]:
data = data['Time Series (Daily)']

Display the data:

In [ ]:
print(data)

* Create a Pandas Dataframe to store data as rows (for each Date) and columns (Date, Open, High, Low and Close)
* Iterate through the JSON file (data structure) and create each row of data for the dataframe

In [ ]:
# stores as pandas datafrome
df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close'])

for key, val in data.items():
    date = dt.datetime.strptime(key, '%Y-%m-%d')
    data_rows = [date.date(), float(
        val['1. open']), float(val['2. high']),
        float(val['3. low']), float(val['4. close'])]

    df.loc[-1, :] = data_rows
    df.index += 1

**Save File**

* specify where the file should be saved
* note that the file is saved as a CSV

In [ ]:
df.to_csv('/content/drive/MyDrive/Datasets/stocks.csv')  

**Forecasting with Prophet**

* https://facebook.github.io/prophet/docs/quick_start.html

Read the CSV file called stocks.csv as a Pandas Dataframe.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/stocks.csv')

* Inspect the first 10 rows of the csv data

In [ ]:
df.head(10)

* Sorting by the date

In [ ]:
df.sort_values(by='Date')

* Obtain the Date and Open stock prices (note that it can be High, Low or Close)
* This is the data that to be used for forecasting i.e. the Open prices of stocks

In [ ]:
data_forecast = df[['Date','Open']]

* Show the  data  for the Date and Open stock prices

In [ ]:
print(data_forecast)

*  Sorting by the date

In [ ]:
data_forecast.sort_values(by='Date')

Facebook's Prophet algorithm requires that the columns have names 'ds', and 'y'
* ds: this will be the date
* y: this is the Open stock price

The 'Date' and 'Open' columns will be renamed to fit with the algorithm's conventions

In [ ]:
data_forecast.rename(columns={'Date':'ds', 'Open':'y'}, inplace=True)

In [ ]:
print(data_forecast)

* Sort by date (now called 'ds')

In [ ]:
data_forecast.sort_values(by='ds', inplace=True)

Create the Prophet model and use it to fit the data

In [ ]:
# create the model for Prophet
model = Prophet()

In [ ]:
# use the created Prophet model to fit (learn) the data
model.fit(data_forecast)

Specify how many days to forecast - call this the future

In [ ]:
future = model.make_future_dataframe(periods=365)

In [ ]:
print(future)

In [ ]:
# now create the forecasts into the future
forecast = model.predict(future)

* Display the results of the forecasts

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

Save our forecasts

In [ ]:
forecast.to_csv('/content/drive/MyDrive/Datasets/forecast.csv')

Create some graphs of the forecasts

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)

In [ ]:
plot_components_plotly(model, forecast)

In [ ]:
forecast.to_csv('/content/drive/MyDrive/Datasets/forecasts_stocks_prophet.csv')